In [1]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup

In [31]:
website_url = requests.get('https://watchstadium.com/news/up-to-date-list-of-college-basketball-players-who-declared-for-2019-nba-draft-04-01-2019/')
soup = BeautifulSoup(website_url.content,'lxml')
soup.prettify()
name_list = soup.find('div',{'class':"post--body"})
name_list
listdata = name_list.findAll('p')
data =[]
count = 0
for cell in listdata:
    count = count+1
    if (count >5):
        data.append(cell.get_text())

In [32]:
import numpy as np
data2 = np.array(data)
players = [items for item in data2 for items in item.split(',')]
del players[1::2]

In [34]:
maindata = []
for name in players:
    realname = name
    name = name.lower()
    name = name.replace(" ", "-")
    website_url = requests.get('https://www.sports-reference.com/cbb/players/'+ name +'-1.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_table = soup.find('table',{'id':'players_per_game'})
    if My_table is not None:
        maindata = maindata + [realname]
        tabledata = My_table.findAll('td')
        data = []
        for cell in tabledata:
            data.append(cell.get_text())
        maindata = maindata +data[-28:]

In [36]:
import numpy as np
maindata2 = np.array(maindata)
refined2 = np.reshape(maindata2, (-1, 29))

In [37]:
tableheader = My_table.findAll('th')
colheaders = []
count = 0
for item in tableheader:
    if (count < 29):
        colheaders.append(item.get_text())
        count=count+1
colheaders.remove('Season')
colheaders = ['Name'] + colheaders
print(colheaders)

['Name', 'School', 'Conf', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '\xa0', 'SOS']


In [38]:
import pandas as pd
college = pd.DataFrame(refined2, columns=colheaders)
college.tail()

,Name,School,Conf,G,GS,MP,FG,FGA,FG%,2P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,,SOS
133,Kris Wilkes,UCLA,,66,65,30.8,5.5,12.6,.436,3.6,...,3.6,4.8,1.7,0.7,0.5,1.6,2.3,15.5,,6.82
134,Emmitt Williams,LSU,,34,8,19.7,2.6,4.3,.614,2.6,...,2.8,5.4,0.4,0.3,0.6,1.1,1.9,7.0,,9.15
135,Grant Williams,Tennessee,,104,101,28.9,5.3,10.2,.516,5.0,...,4.0,6.5,2.1,0.9,1.5,2.1,3.2,15.7,,10.15
136,Zion Williamson,Duke,,33,33,30.0,9.0,13.2,.680,8.2,...,5.4,8.9,2.1,2.1,1.8,2.4,2.1,22.6,,11.97
137,Kenny Wooten,Oregon,,70,38,21.8,2.6,4.1,.634,2.6,...,3.3,4.7,0.5,0.5,2.4,1.1,2.6,6.4,,5.49


In [40]:
college = college.drop(['\xa0', 'Conf'], axis=1)

In [47]:
college.head()

,School,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,SOS
Name,,,,,,,,,,,,,,,,,,,,,
Bryce Aiken,Harvard,58,34,29.4,5.0,12.4,0.406,2.9,6.1,0.468,...,0.4,2.1,2.4,2.8,0.8,0.0,2.8,1.9,16.8,-1.09
Nickeil Alexander-Walker,Virginia Tech,67,67,29.9,4.7,10.1,0.464,3.0,5.6,0.529,...,0.6,3.4,4.0,2.7,1.4,0.5,2.2,1.9,13.5,7.25
Al-Wajid Aminu,North Florida,34,26,26.0,3.6,6.7,0.542,3.4,5.9,0.578,...,2.6,3.8,6.5,0.7,0.4,1.1,2.1,3.6,9.1,-1.65
Desmond Bane,TCU,109,82,28.7,4.2,8.2,0.516,2.8,4.8,0.578,...,0.8,3.4,4.2,1.9,0.8,0.3,1.3,2.2,11.5,10.08
RJ Barrett,Duke,38,38,35.3,8.4,18.5,0.454,6.5,12.2,0.529,...,1.6,6.0,7.6,4.3,0.9,0.4,3.2,1.8,22.6,11.97


In [42]:
col = college.columns
cols = []
for co in col:
    cols.append(co)
cols.remove('Name')
cols.remove('School')
print(cols)

['G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'SOS']


In [43]:
for col in cols:
    college[col] = pd.to_numeric(college[col], errors='coerce')
college = college.dropna()
college.head()

,Name,School,G,GS,MP,FG,FGA,FG%,2P,2PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,SOS
1,Bryce Aiken,Harvard,58,34,29.4,5.0,12.4,0.406,2.9,6.1,...,0.4,2.1,2.4,2.8,0.8,0.0,2.8,1.9,16.8,-1.09
2,Nickeil Alexander-Walker,Virginia Tech,67,67,29.9,4.7,10.1,0.464,3.0,5.6,...,0.6,3.4,4.0,2.7,1.4,0.5,2.2,1.9,13.5,7.25
3,Al-Wajid Aminu,North Florida,34,26,26.0,3.6,6.7,0.542,3.4,5.9,...,2.6,3.8,6.5,0.7,0.4,1.1,2.1,3.6,9.1,-1.65
4,Desmond Bane,TCU,109,82,28.7,4.2,8.2,0.516,2.8,4.8,...,0.8,3.4,4.2,1.9,0.8,0.3,1.3,2.2,11.5,10.08
5,RJ Barrett,Duke,38,38,35.3,8.4,18.5,0.454,6.5,12.2,...,1.6,6.0,7.6,4.3,0.9,0.4,3.2,1.8,22.6,11.97


In [46]:
college = college.set_index('Name')

In [48]:
college.to_csv('2019DraftClass.csv')